In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../src')

In [3]:
from parse_cesar1 import get_timestep_tuples
import pickle

In [4]:
from pyprocessta.causalimpact import _select_unrelated_x
from pyprocessta.model.tcn import transform_data, run_model

In [5]:
from darts import TimeSeries
import pandas as pd
from definitions import TARGETS
import matplotlib.pyplot as plt

MEAS_COLUMNS = [
    "TI-19",
    "FI-16",
    "TI-33",
    "FI-2",
    "FI-151",
    "TI-8",
    "FI-241",
    "valve-position-12",  # dry-bed
    "FI-38",  # stripper
    "PI-28",  # stripper
    "TI-28",  # stripper
    "FI-20",
    "FI-30",
    "TI-3",
    "FI-19",
    "FI-211",
    "FI-11",
    "TI-30",
    "PI-30",
    "TI-1213",
    "TI-4",
    "FI-23",
    "delta_t",
]

In [6]:
df = pd.read_pickle('detrended.pkl')
df = df.dropna()
df['delta_t'] = df['TI-35'] - df["TI-4"]
MEAS_COLUMNS.append('delta_t')

In [7]:
plt.plot(df['delta_t'])

In [8]:
with open('step_times.pkl', 'rb') as handle:
    times = pickle.load(handle)

In [9]:
a, b = get_timestep_tuples(df, times, 0)

In [10]:
a

[Timestamp('2020-01-15 11:44:01', freq='2T'),
 Timestamp('2020-01-15 14:08:01', freq='2T')]

In [11]:
def get_causalimpact_splits(x, y, day, times, df): 
    a, b = get_timestep_tuples(df, times, day)
 
    x_way_before, x_after = x.split_before(a[1])
    y_way_before, y_after = y.split_before(pd.Timestamp(a[1]))

    _, x_before = x_way_before.split_before(a[0])
    _, y_before = y_way_before.split_before(a[0])

    x_during, x_test = x_after.split_before(b[1])
    y_during, y_test = y_after.split_before(b[1])

    return (x_before, y_before), (x_during, y_during), (x_test, y_test)

    
    

In [12]:
y = TimeSeries.from_dataframe(df, value_cols=TARGETS)
x = TimeSeries.from_dataframe(df, value_cols=MEAS_COLUMNS)

In [13]:
before, during, after = get_causalimpact_splits(x, y, 1, times, df)

In [14]:
train_tuple, test_tuple, scalers = transform_data(before, [during, after])

In [15]:
run_model(train_tuple)

In [16]:
run_model(train_tuple)